In [1]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import time
import importlib
from pathlib import Path
from datetime import datetime

from numba import jit
import numpy as np
import pandas as pd

# import json
# import yaml
from pprint import pprint


# required to enable non-blocking interaction:
# %gui qt
# $env:QT_API="pyqt6"
%gui qt5
# from PyQt5.Qt import QApplication
# # start qt event loop
# _instance = QApplication.instance()
# if not _instance:
#     _instance = QApplication([])
# app = _instance
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph.opengl as gl # for 3D raster plot

# ## Panel:
import param
# import panel as pn
# from panel.interact import interact, interactive, fixed, interact_manual
# from panel.viewable import Viewer
# pn.extension()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import get_arguments_as_optional_dict, inspect_callable_arguments
from pyphocorehelpers.function_helpers import compose_functions
from pyphocorehelpers.indexing_helpers import partition, build_spanning_bins, compute_spanning_bins, compute_position_grid_size, compute_paginated_grid_config
from pyphocorehelpers.print_helpers import PrettyPrintable, WrappingMessagePrinter
from pyphocorehelpers.geometry_helpers import compute_data_extent, compute_data_aspect_ratio, corner_points_from_extents
from pyphocorehelpers.DataStructure.dynamic_parameters import DynamicParameters
from pyphocorehelpers.performance_timing_helpers import WrappingPerformanceTimer
from pyphocorehelpers.gui.interaction_helpers import CallbackWrapper

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.SessionSelectionAndFiltering import batch_filter_session, build_custom_epochs_filters
from pyphoplacecellanalysis.General.KnownDataSessionTypeProperties import KnownDataSessionTypeProperties
from pyphoplacecellanalysis.General.Pipeline.Stages.Display import DefaultDisplayFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.Ratemaps import DefaultRatemapDisplayFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import DefaultDecoderDisplayFunctions
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster import Spike3DRaster

from pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_RasterPlot import plot_raster_plot, _display_pyqtgraph_raster_plot

# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core

    importlib.reload(core)
except ImportError:
    sys.path.append(r"C:\Users\Pho\repos\NeuroPy")  # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print("neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.")
    from neuropy import core

# Neuropy:
from neuropy.core.session.data_session_loader import DataSessionLoader
from neuropy.core.session.dataSession import DataSession
from neuropy.core.laps import Laps
from neuropy.analyses.placefields import PlacefieldComputationParameters, perform_compute_placefields
from neuropy.analyses.laps import estimation_session_laps
from neuropy.core.epoch import NamedTimerange

from neuropy.analyses.placefields import PlacefieldComputationParameters, perform_compute_placefields
from neuropy.core.neuron_identities import NeuronIdentity, build_units_colormap, PlotStringBrevityModeEnum
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_spike_counts, debug_print_subsession_neuron_differences
from neuropy.plotting.ratemaps import enumTuningMap2DPlotVariables
# from neuropy.utils.mixins.time_slicing import verify_non_overlapping, add_PBE_identity
from neuropy.utils.efficient_interval_search import get_non_overlapping_epochs, drop_overlapping

known_data_session_type_dict = {'kdiba':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.kdiba_old_format_session(a_base_dir)),
                               basedir=Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')),
                'bapun':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.bapun_data_session(a_base_dir)),
                               basedir=Path('R:\data\Bapun\Day5TwoNovel'))
               }
known_data_session_type_dict['kdiba'].post_load_functions = [lambda a_loaded_sess: estimation_session_laps(a_loaded_sess)]

# known_data_session_type_dict['kdiba'].name
# from matplotlib.figure import Figure
# import matplotlib.pyplot as plt
# %matplotlib qt
# from matplotlib.transforms import IdentityTransform

enable_saving_to_disk = False
# common_parent_foldername = Path(r'R:\Dropbox (Personal)\Active\Kamran Diba Lib\Pho-Kamran-Meetings\Final Placemaps 2021-01-14')
common_parent_foldername = Path(r'R:\Dropbox (Personal)\Active\Kamran Diba Lib\Pho-Kamran-Meetings\2022-01-16')

def compute_position_grid_bin_size(x, y, num_bins=(64,64), debug_print=False):
    """ Compute Required Bin size given a desired number of bins in each dimension
    Usage:
        active_grid_bin = compute_position_grid_bin_size(curr_kdiba_pipeline.sess.position.x, curr_kdiba_pipeline.sess.position.y, num_bins=(64, 64)
    """
    out_grid_bin_size, out_bins, out_bins_infos = compute_position_grid_size(x, y, num_bins=num_bins)
    active_grid_bin = tuple(out_grid_bin_size)
    if debug_print:
        print(f'active_grid_bin: {active_grid_bin}') # (3.776841861770752, 1.043326930905373)
    return active_grid_bin

# WARNING! TODO: Changing the smooth values from (1.5, 1.5) to (0.5, 0.5) was the difference between successful running and a syntax error!
# try:
#     active_grid_bin
# except NameError as e:
#     print('setting active_grid_bin = None')
#     active_grid_bin = None
# finally:
#     # active_session_computation_config = PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=active_grid_bin, smooth=(1.0, 1.0), frate_thresh=0.2, time_bin_size=0.5) # if active_grid_bin is missing, figure out the name
#     active_session_computation_config = PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=active_grid_bin, smooth=(1.0, 1.0), frate_thresh=0.2, time_bin_size=0.5) # if active_grid_bin is missing, figure out the name

## Dynamic mode:
def _build_active_computation_configs(sess):
    """ _get_computation_configs(curr_kdiba_pipeline.sess)
        # From Diba:
        # (3.777, 1.043) # for (64, 64) bins
        # (1.874, 0.518) # for (128, 128) bins

    """
    # active_grid_bin = compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(64, 64))
    # active_session_computation_config.computation_epochs = None # set the placefield computation epochs to None, using all epochs.
    # return [PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(64, 64)), smooth=(1.0, 1.0), frate_thresh=0.2, time_bin_size=0.5, computation_epochs = None)]
    # return [PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(128, 128)), smooth=(2.0, 2.0), frate_thresh=0.2, time_bin_size=0.5, computation_epochs = None)]
    return [PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(64, 64)), smooth=(2.0, 2.0), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None)]
    # return [PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(3.777, 1.043), smooth=(1.0, 1.0), frate_thresh=0.2, time_bin_size=0.5, computation_epochs = None)]

    # return [PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(32, 32)), smooth=(1.0, 1.0), frate_thresh=0.2, time_bin_size=0.5, computation_epochs = None),
    #         PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(64, 64)), smooth=(1.0, 1.0), frate_thresh=0.2, time_bin_size=0.5, computation_epochs = None),
    #         PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(128, 128)), smooth=(1.0, 1.0), frate_thresh=0.2, time_bin_size=0.5, computation_epochs = None),
    #        ]



## KDiba Format:

In [2]:
## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m
# From pre-computed .mat files:
## 07: 
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'
# # ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'
# curr_kdiba_pipeline = NeuropyPipeline(name='kdiba_pipeline', session_data_type='kdiba', basedir=known_data_session_type_dict['kdiba'].basedir, load_function=known_data_session_type_dict['kdiba'].load_function)
curr_kdiba_pipeline = NeuropyPipeline.init_from_known_data_session_type('kdiba', known_data_session_type_dict['kdiba'])
# active_grid_bin = compute_position_grid_bin_size(curr_kdiba_pipeline.sess.position.x, curr_kdiba_pipeline.sess.position.y, num_bins=(64, 64))
# active_session_computation_config.grid_bin = active_grid_bin
active_session_computation_configs = _build_active_computation_configs(curr_kdiba_pipeline.sess)

def build_pyramidal_epochs_filters(sess):
    sess.epochs.t_start = 22.26 # exclude the first short period where the animal isn't on the maze yet
    active_session_filter_configurations = {'maze1': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(x.epochs.get_named_timerange('maze1')), x.epochs.get_named_timerange('maze1')),
                                        'maze2': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(x.epochs.get_named_timerange('maze2')), x.epochs.get_named_timerange('maze2')),
                                        'maze': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]])), NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]]))
                                       }
    return active_session_filter_configurations

active_session_filter_configurations = build_pyramidal_epochs_filters(curr_kdiba_pipeline.sess)

is_non_overlapping_lap = get_non_overlapping_epochs(curr_kdiba_pipeline.sess.laps.to_dataframe()[['start','stop']].to_numpy())
only_good_laps_df = curr_kdiba_pipeline.sess.laps.to_dataframe()[is_non_overlapping_lap]
curr_kdiba_pipeline.sess.laps = Laps(only_good_laps_df) # replace the laps object with the filtered one

lap_specific_epochs = curr_kdiba_pipeline.sess.laps.as_epoch_obj()
any_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(len(curr_kdiba_pipeline.sess.laps.lap_id))])
even_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(0, len(curr_kdiba_pipeline.sess.laps.lap_id), 2)])
odd_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(1, len(curr_kdiba_pipeline.sess.laps.lap_id), 2)])

# Copy the active session_computation_config:
for i in np.arange(len(active_session_computation_configs)):
    active_session_computation_configs[i].computation_epochs = any_lap_specific_epochs # add the laps epochs to all of the computation configs.

curr_kdiba_pipeline.filter_sessions(active_session_filter_configurations)
curr_kdiba_pipeline.perform_computations(active_session_computation_configs[0]) # Causes "IndexError: index 59 is out of bounds for axis 0 with size 59"
curr_kdiba_pipeline.prepare_for_display() # TODO: pass a display config
# set curr_active_pipeline for testing:
curr_active_pipeline = curr_kdiba_pipeline

basedir is already Path object.
	 basepath: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53
	 session_name: 2006-6-07_11-26-53
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikes.mat... done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position.npy... 2006-6-07_11-26-53.position.npy saved
done.
	 Failure loading .interpolated_spike_positions.npy. Must recompute.

	 Saving updated interpolated spike position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.interpolated_spike_positions.npy.

c:\users\pho\repos\neuropy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
c:\users\pho\repos\neuropy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
c:\users\pho\repos\neuropy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Applying session filter named "maze1"...
Constraining to units with type: pyramidal
Constraining to epoch with times (start: 22.26, end: 1739.1533641185379)
computing neurons mua for session...



c:\users\pho\repos\neuropy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
c:\users\pho\repos\neuropy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
c:\users\pho\repos\neuropy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Applying session filter named "maze2"...
Constraining to units with type: pyramidal
Constraining to epoch with times (start: 1739.1533641185379, end: 1932.4200048116618)
computing neurons mua for session...

Applying session filter named "maze"...
Constraining to units with type: pyramidal


...

c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\Analysis\reconstruction.py:217: RuntimeWarning: divide by zero encountered in true_divide
  return C * np.exp(numerator/denominator)


Performing single_computation on filtered_session with filter named "maze2"...
Recomputing active_epoch_placefields... pre speed filtering: 545 spikes.
post speed filtering: 314 spikes.
	 done.
Recomputing active_epoch_placefields2D... pre speed filtering: 545 spikes.
post speed filtering: 329 spikes.
	 done.
Performing perform_registered_computations(...) with 6 registered_computation_functions...
Performing single_computation on filtered_session with filter named "maze"...
Recomputing active_epoch_placefields... pre speed filtering: 20747 spikes.
post speed filtering: 13582 spikes.


c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\Analysis\reconstruction.py:177: RuntimeWarning: divide by zero encountered in double_scalars
  C_tau_n = 1.0 / np.sum(un_normalized_result) # normalize the result
c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\Analysis\reconstruction.py:178: RuntimeWarning: invalid value encountered in multiply
  result = C_tau_n * un_normalized_result
c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\Analysis\reconstruction.py:217: RuntimeWarning: divide by zero encountered in true_divide
  return C * np.exp(numerator/denominator)


	 done.
Recomputing active_epoch_placefields2D... pre speed filtering: 20747 spikes.
post speed filtering: 13858 spikes.
	 done.
Performing perform_registered_computations(...) with 6 registered_computation_functions...


c:\users\pho\repos\pyphoplacecellanalysis\src\pyphoplacecellanalysis\Analysis\reconstruction.py:217: RuntimeWarning: divide by zero encountered in true_divide
  return C * np.exp(numerator/denominator)


In [3]:
## Testing burst detection
from neurodsp.burst import detect_bursts_dual_threshold, compute_burst_stats
from neurodsp.plts.time_series import plot_time_series, plot_bursts # for plotting results

# curr_active_pipeline.sess.spikes_df.spikes.neuron_ids

# grouped = curr_active_pipeline.sess.spikes_df.groupby('aclu')

# neuron_split_spike_dfs = [spk_df.groupby('aclu').get_group(neuron_id)[['t','x','y','lin_pos']] for neuron_id in active_epoch_session.neuron_ids] # dataframes split for each ID:
# neuron_split_spike_dfs = curr_active_pipeline.sess.spikes_df.spikes.get_split_by_unit()

# grouped['t_rel_seconds'].diff()

# .diff()
# curr_active_pipeline.sess.spikes_df

In [4]:
## 2D Spike Train Visualization
# import pyqtgraph.opengl as gl # for 3D raster plot
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterBase import SpikeRasterBase
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster import Spike3DRaster

curr_epoch_name = 'maze1'
curr_epoch = curr_active_pipeline.filtered_epochs[curr_epoch_name] # <NamedTimerange: {'name': 'maze1', 'start_end_times': array([  22.26      , 1739.15336412])};>
curr_sess = curr_active_pipeline.filtered_sessions[curr_epoch_name]
curr_spikes_df = curr_sess.spikes_df
curr_computations_results = curr_active_pipeline.computation_results[curr_epoch_name]

In [ ]:
# pp = pprint.PrettyPrinter(depth=4)
# pp.pprint(curr_computations_results.computed_data)
# print(yaml.dump(curr_computations_results.computed_data, default_flow_style=False))
# print(json.dumps(curr_computations_results.computed_data, indent=2, default=str))



In [ ]:
# curr_computations_results.computed_data.keys()

# from pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_DataTreeWidget import plot_dataTreeWidget

# tree, app = plot_dataTreeWidget(data=curr_computations_results.computed_data, title='PhoOutputDataTreeApp')  
# # curr_computations_results.computed_data.keys()

# # tree = pg.DataTreeWidget(data=curr_computations_results.computed_data)
# tree.show()
# pg.exec()

# Independent 2D Raster Plotkeys

In [ ]:
spike_raster_plt_2d = Spike2DRaster(curr_spikes_df, window_duration=15.0 * 60.0, window_start_time=30.0, neuron_colors=None)

# Independent 3D Raster Plot

In [ ]:
spike_raster_plt_3d = Spike3DRaster(curr_spikes_df, window_duration=15.0, window_start_time=30.0, neuron_colors=None)

In [ ]:
spike_raster_plt_3d.add_render_epochs(curr_sess.pbe.starts, curr_sess.pbe.durations, epoch_type_name='PBE')

In [ ]:
spike_raster_plt_3d.RenderTimeEpochMeshesMixin_on_update_window()

# 3D Raster Plot Custom Controlled by 2D Raster Plot

In [6]:
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
def _move_widget_to_top_left_corner(widget, screen_index:int=0, debug_print:bool=False):
    widget = widget.app.desktop()
    assert screen_index <= (widget.screenCount()-1), f"specified screen_index screen_index: {screen_index} is invalid. widget.screenCount(): {widget.screenCount()}"
    secondary_screen_rect = widget.screenGeometry(screen_index) # get screen geometry of secondary screen. PyQt5.QtCore.QRect(-3440, 0, 3440, 1440)
    if debug_print:
        print(f'secondary_screen_rect: {secondary_screen_rect}')
        print(f'widget: {widget}')
    # secondary_screen_rect.topLeft() # PyQt5.QtCore.QPoint(-3440, 0)
    # Move to top-left corner of secondary screen:
    widget.move(secondary_screen_rect.topLeft())
    # spike_raster_plt_3d.move(-secondary_screen_rect.width(), secondary_screen_rect.y())
    
def _align_3d_and_2d_windows(spike_raster_plt_3d, spike_raster_plt_2d, debug_print=False):
    """ align the two separate windows (for the 3D Raster Plot and the 2D Raster plot that controls it)
        Usage:
            _align_3d_and_2d_windows(spike_raster_plt_3d, spike_raster_plt_2d)
    """
    # _move_widget_to_top_left_corner(spike_raster_plt_3d, screen_index=1, debug_print=debug_print) # move to secondary screen's top-left corner.
    geom = spike_raster_plt_3d.window().geometry() # get the QTCore PyRect object
    if debug_print:
        print(f'geom.bottom: {geom.bottom()}') # geom.bottom: 941
    x,y,dx,dy = geom.getRect()
    if debug_print:
        print(f'geom: {geom}') # geom: PyQt5.QtCore.QRect(-3354, 176, 1920, 900)
    # after moving window down on screen: geom: PyQt5.QtCore.QRect(-3362, 478, 1920, 900)
    # THEREFORE larger y values are down
    desired_2d_window_y = y + dy # place directly below 3D window
    # desired_2d_window_y = y
    if debug_print:
        print(f'y: {y}, dy: {dy}, desired_2d_window_y: {desired_2d_window_y}')
    spike_raster_plt_2d.window().setGeometry(x, desired_2d_window_y, dx, dy*0.5)

In [7]:
def build_spike_3d_raster_with_2d_controls(curr_spikes_df):
    spike_raster_plt_3d = Spike3DRaster(curr_spikes_df, window_duration=15.0, window_start_time=30.0, neuron_colors=None)
    # Connect the 2D window scrolled signal to the 3D plot's spikes_window.update_window_start_end function
    spike_raster_plt_2d = Spike2DRaster(curr_spikes_df, window_duration=15.0, window_start_time=30.0, neuron_colors=None)
    spike_3d_to_2d_window_connection = spike_raster_plt_2d.window_scrolled.connect(spike_raster_plt_3d.spikes_window.update_window_start_end)
    spike_raster_plt_3d.disable_render_window_controls()
    _align_3d_and_2d_windows(spike_raster_plt_3d, spike_raster_plt_2d) # Align the two windows
    return spike_raster_plt_3d, spike_raster_plt_2d, spike_3d_to_2d_window_connection


spike_raster_plt_3d, spike_raster_plt_2d, spike_3d_to_2d_window_connection = build_spike_3d_raster_with_2d_controls(curr_spikes_df)

Spike2DRaster.setup(): adding "visualization_raster_y_location" column to spikes_df...
done.


C:\Users\pho\AppData\Local\Temp\ipykernel_2200\599548591.py:32: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  spike_raster_plt_2d.window().setGeometry(x, desired_2d_window_y, dx, dy*0.5)


In [ ]:
_move_widget_to_top_left_corner(spike_raster_plt_3d, screen_index=1, debug_print=True)
# _align_3d_and_2d_windows(spike_raster_plt_3d, spike_raster_plt_2d, debug_print=True) # Align the two windows    

# tRectangle = self.frameGeometry()

In [8]:
spike_3d_animation_to_2d_scroll_ctrl_connection = spike_raster_plt_3d.animation_active_time_window.window_changed_signal.connect(spike_raster_plt_2d.update_scroll_window_region)
# spike_raster_plt_3d.animationThread.update_signal.connect(spike_raster_plt_2d.update_scroll_window_region)

In [ ]:
desktopWidget = QtWidgets.QDesktopWidget()
desktopRect = desktopWidget.availableGeometry() # PyQt5.QtCore.QRect(0, 0, 3570, 2120)
# desktopRect.topLeft().x()
# desktopRect.topLeft().y()
currWindowRect = spike_raster_plt_3d.frameGeometry()
print(f'currWindowRect: {currWindowRect}')
# currWindowRect.moveCenter(centerPoint)
currWindowRect.moveTopLeft(desktopRect.topLeft())
spike_raster_plt_3d.move(currWindowRect.topLeft())


spike_raster_plt_3d.move(currWindowRect.topLeft()) # currWindowRect: PyQt5.QtCore.QRect(0, 0, 1922, 932)

# centerPoint = QtWidgets.QDesktopWidget().availableGeometry().center()
# qtRectangle.moveCenter(centerPoint)
# self.move(qtRectangle.topLeft())

# Testing/Playground

In [ ]:
curr_sess.mua.t_start # 22.26
curr_sess.mua.t_stop # 1739.153

# curr_sess.mua.time
# np.shape(curr_sess.mua.firing_rate) # (1716893,)
# np.shape(curr_sess.mua.spike_counts) # (1716893,)

smoothed_mua = curr_sess.mua.get_smoothed()
smoothed_mua.to_dataframe()

In [ ]:
# curr_sess.config
curr_sess.

### Test adding the MUA firing rate curve to the 3D plot

In [ ]:
linear_position_plot_data = curr_sess.position.to_dataframe()[['t', 'lin_pos']]
spike_raster_plt_3d.add_3D_time_curves(linear_position_plot_data)

# curr_sess.position.linear_pos


In [ ]:
# plot_data = pd.DataFrame({'t': curr_sess.mua.time, 'mua_firing_rate': curr_sess.mua.firing_rate, 'mua_spike_counts': curr_sess.mua.spike_counts})
plot_data = pd.DataFrame({'t': curr_sess.mua.time, 'mua_firing_rate': curr_sess.mua.firing_rate})
spike_raster_plt_3d.add_3D_time_curves(plot_data)




# # get all non-time columns
# data_column_names = np.setdiff1d(plot_data.columns, np.array(['t']))
# data_column_names

# plot_data[data_column_names]

# plot_datasource = CurveDatasource(plot_data)

In [ ]:
plot_UIDs = spike_raster_plt_3d.params.time_curves_datasource.datasource_UIDs # ['default_plot_datasource.mua_firing_rate']
spike_raster_plt_3d.plots.time_curves[plot_UIDs[0]]

In [ ]:
spike_raster_plt_3d.detach_3d_time_curves_datasource()

In [ ]:
## Update:
curr_plot3D_active_window_data = plot_datasource.get_updated_data_window(spike_raster_plt_3d.spikes_window.active_window_start_time, spike_raster_plt_3d.spikes_window.active_window_end_time) # get updated data for the active window from the datasource
curve_y_value = -spike_raster_plt_3d.n_half_cells
curr_x = DataSeriesToSpatial.temporal_to_spatial_map(curr_plot3D_active_window_data['t'].to_numpy(), spike_raster_plt_3d.spikes_window.active_window_start_time, spike_raster_plt_3d.spikes_window.active_window_end_time, spike_raster_plt_3d.temporal_axis_length, center_mode='zero_centered')
pts = np.column_stack([curr_x, np.full_like(curr_x, curve_y_value), curr_plot3D_active_window_data['mua_firing_rate'].to_numpy()])
plt.setData(pos=pts)

In [ ]:
curr_plot3D_active_window_data.plot()

## Debugging Rendering:

In [ ]:
# -self.n_half_cells
debug_point_pos = (-spike_raster_plt_3d.half_temporal_axis_length, -spike_raster_plt_3d.n_half_cells, 0.0) # (-10.69215668859044, -20.0, 0.0)
debug_point_pos

In [ ]:
spike_raster_plt_3d.ui.gl_test_points = []
md = gl.MeshData.sphere(rows=4, cols=4)
m1 = gl.GLMeshItem(meshdata=md, smooth=False, drawFaces=False, drawEdges=True, edgeColor=(0.7,0.7,0.7,0.8))
m1.translate(debug_point_pos[0], debug_point_pos[1], debug_point_pos[2])
m1.scale(0.5, 0.25, 0.25)
m1.setGLOptions('additive')
spike_raster_plt_3d.ui.main_gl_widget.addItem(m1)
spike_raster_plt_3d.ui.gl_test_points.append(m1)


# spike_raster_plt_3d.dynamic_add_widget()

In [ ]:
# REMOVE

spike_raster_plt_3d.ui.main_gl_widget.removeItem(m1)
spike_raster_plt_3d.ui.gl_test_points.remove(m1)

In [ ]:
spike_raster_plt_3d.add_render_epochs(curr_sess.pbe.starts, curr_sess.pbe.durations, epoch_type_name='PBE')

In [ ]:
spike_raster_plt_3d._build_epoch_meshes(curr_sess.pbe.starts, curr_sess.pbe.durations)

# spike_raster_plt_3d.ui.parent_epoch_container_item.setObjectName('container')
# spike_raster_plt_3d.ui.parent_epoch_container_item.childItems()
# curr_cube.translate(t_centers[i], 0, 0)
# curr_cube.scale(durations[i], 1, 0.25)
    

In [ ]:
spike_raster_plt_3d._remove_epoch_meshes()

In [ ]:
# for (i, aCube) in enumerate(spike_raster_plt_3d.ui.new_cube_objects):
#     aCube.setParent(None)
#     aCube.deleteLater()

from vispy import geometry

# spike_raster_plt_3d.ui.new_cube_objects.clear()
# spike_raster_plt_3d.ui.new_cube_objects
print(f'number of cube objects: {len(spike_raster_plt_3d.ui.new_cube_objects)}') # number of cube objects: 206

spike_raster_plt_3d._temporal_to_spatial(curr_sess.pbe.starts)

# vispy.geometry.create_box(width=1, height=1, depth=1, width_segments=1, height_segments=1, depth_segments=1, planes=None)
box_mesh_data = geometry.create_box(width=1, height=1, depth=1, width_segments=1, height_segments=1, depth_segments=1, planes=None)
# vispy.geometry.create_plane(width=1, height=1, width_segments=1, height_segments=1, direction='+z')
plane_mesh_data = geometry.create_plane(width=1, height=1, width_segments=1, height_segments=1, direction='+z')
# vertices, faces, outline

plane_mesh_data

In [ ]:
spike_raster_plt_3d.update_epoch_meshes(curr_sess.pbe.starts, curr_sess.pbe.durations)

In [ ]:
spike_raster_plt_2d.render_window_duration # 15.0
spike_raster_plt_2d.spikes_window.total_df_start_end_times # (22.30206346133491, 1739.1355703625595)
spike_raster_plt_2d.spikes_window.active_window_start_time # 30.0
spike_raster_plt_2d.spikes_window.active_window_end_time # 45.0

In [ ]:
spike_raster_plt_3d.close_signal.connect(spike_raster_plt_2d.closeEvent)

### Align 3d window and the 2d window that controls it

In [ ]:



# # to put it into the upper left corner for example:
# mngr.window.setGeometry(50,100,640, 545)
# # If one doesn't know the x- and y-width one can read them out first, like so:
# # get the QTCore PyRect object
# geom = mngr.window.geometry()
# x,y,dx,dy = geom.getRect()
# # and then set the new position with the same size:
# mngr.window.setGeometry(newX, newY, dx, dy)

In [ ]:
# spike_raster_plt.ui.main_plot_widget.setXRange(0.0, +spike_raster_plt.render_window_duration)

# spike_raster_plt.ui.scatter_plot.enableAutoRange('xy', False)  ## stop auto-scaling after the first data set is plotted
# spike_raster_plt.ui.scatter_plot.dataBounds()
# spike_raster_plt.ui.scatter_plot.flags()
# spike_raster_plt.ui.scatter_plot.setPointsVisible
# spike_raster_plt.ui.main_plot_widget.addScrollBarWidget()

# print(f'{spike_raster_plt.ui.main_plot_widget.contentsMargins()}')
# spike_raster_plt.ui.scatter_plot.points()

# spike_raster_plt.ui.main_plot_widget.setXRange
# spike_raster_plt.ui.main_plot_widget.setLimits()
# spike_raster_plt.ui.main_plot_widget # PlotItem

min_x, max_x = spike_raster_plt_2d.ui.scroll_window_region.getRegion() # (59.62061245756003, 76.83228787177144)
scroll_window_width = max_x - min_x
spike_raster_plt_2d_old_time_window = spike_raster_plt_2d.spikes_window.active_time_window # (30.0, 930.0)
print(f'spike_raster_plt_2d: min_x: {min_x}, max_x: {max_x}, scroll_window_width: {scroll_window_width}, spike_raster_plt_2d_old_time_window: {spike_raster_plt_2d_old_time_window}') # spike_raster_plt_2d: min_x: 111.72696109457753, max_x: 149.92210657285403, scroll_window_width: 38.1951454782765, spike_raster_plt_2d_old_time_window: (111.72696109457753, 149.92210657285403)

# def _debug_print_spike_raster_plot_window(
spike_raster_plt_3d_old_time_window = spike_raster_plt_3d.spikes_window.active_time_window # (30.0, 930.0)
print(f'spike_raster_plt_3d: spike_raster_plt_3d_old_time_window: {spike_raster_plt_3d_old_time_window}') # spike_raster_plt_3d: spike_raster_plt_3d_old_time_window: (47.599999999999625, 62.599999999999625)

# Update the 3D Time window from the 2D one:
# spike_raster_plt_3d.spikes_window.active_time_window = spike_raster_plt_2d_old_time_window
spike_raster_plt_3d.spikes_window.update_window_start_end(spike_raster_plt_2d_old_time_window[0], spike_raster_plt_2d_old_time_window[1])

# spike_raster_plt.render_window_duration = 
# spike_raster_plt.spikes_window
# spike_raster_plt.temporal_zoom_factor
# # self.spikes_window.update_window_start(next_start_timestamp)

# spike_raster_plt.ui.spinTemporalZoomFactor.setValue(1.0)
# spike_raster_plt.ui.spinRenderWindowDuration.setValue(scroll_window_width)
# spike_raster_plt.spikes_window.update_window_start(min_x)

# curr_plot_item = spike_raster_plt.ui.main_plot_widget.getPlotItem()
# curr_view_box = curr_plot_item.getViewBox()
# # curr_view_box.autoRangeEnabled()
# # curr_view_box.setAutoPan(False)

# curr_view_box.setDefaultPadding(0.0)
# spike_raster_plt.ui.main_plot_widget.viewRect()
# spike_raster_plt.ui.scatter_plot.opts
# spike_raster_plt._update_plot_ranges()

In [ ]:
# @pyqtSlot(float, float)
def _test_on_2d_window_scrolled(start_v, end_v):
    print(f'_test_on_2d_window_scrolled(start_v: {start_v}, end_v: {end_v})')
    spike_raster_plt_3d.spikes_window.update_window_start_end(start_v, end_v)

spike_raster_plt_2d.window_scrolled.connect(_test_on_2d_window_scrolled)

In [ ]:
spike_raster_plt_3d.ui.spinRenderWindowDuration.setValue(scroll_window_width)

In [ ]:
spike_raster_plt_3d.ui.spinTemporalZoomFactor.setValue(10.0)

In [ ]:
spike_raster_plt_3d.update()

In [ ]:
lower_edge = spike_raster_plt.y / spike_raster_plt.n_cells
upper_edge = (spike_raster_plt.y + 1.0)/spike_raster_plt.n_cells

print(f'lower_edge: {lower_edge}\n upper_edge: {upper_edge}')

In [ ]:
lower_y = DataSeriesToSpatial.build_series_identity_axis(spike_raster_plt.n_cells, center_mode=spike_raster_plt.params.center_mode, bin_position_mode='left_edges', side_bin_margins = spike_raster_plt.params.side_bin_margins) / spike_raster_plt.n_cells
upper_y = DataSeriesToSpatial.build_series_identity_axis(spike_raster_plt.n_cells, center_mode=spike_raster_plt.params.center_mode, bin_position_mode='right_edges', side_bin_margins = spike_raster_plt.params.side_bin_margins) / spike_raster_plt.n_cells

print(f'lower_y: {lower_y}\n upper_y: {upper_y}')

In [ ]:
## 3D Spike Train Visualization
# import pyqtgraph.opengl as gl # for 3D raster plot
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Spike3DRaster import Spike3DRaster

# importlib.reload(pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_RasterPlot)

curr_epoch_name = 'maze1'
curr_epoch = curr_active_pipeline.filtered_epochs[curr_epoch_name] # <NamedTimerange: {'name': 'maze1', 'start_end_times': array([  22.26      , 1739.15336412])};>
curr_sess = curr_active_pipeline.filtered_sessions[curr_epoch_name]
curr_spikes_df = curr_sess.spikes_df

spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=1.0, window_start_time=30.0, neuron_colors=None)
# spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=4.0, window_start_time=30.0, neuron_colors=None)
# spike_raster_plt = Spike3DRaster(curr_spikes_df, window_duration=0.2, window_start_time=30.0)

# spike_raster_plt.animation()
# spike_raster_plt = None

In [ ]:
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')
# spike_raster_plt.spikes_window.active_window_start_time = 50.0
spike_raster_plt.spikes_window.update_window_start(90.0)
print(f'spike_raster_plt.spikes_window.active_time_window: {spike_raster_plt.spikes_window.active_time_window}')

In [ ]:
spike_raster_plt.animation_time_step # 0.03 (seconds)

In [ ]:
# spike_raster_plt.render_window_duration # 4.0 (seconds)
# spike_raster_plt.spikes_window.window_duration = 1.0

spike_raster_plt.temporal_zoom_factor # 40.0
spike_raster_plt.temporal_zoom_factor = 80.0

In [ ]:
spike_raster_plt.on_adjust_temporal_spatial_mapping()

In [ ]:
# curr_transform = spike_raster_plt.ui.gz.transform() # PyQt5.QtGui.QMatrix4x4(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, -10.0, 0.0, 0.0, 0.0, 1.0)
curr_transform = spike_raster_plt.ui.gx.transform() # PyQt5.QtGui.QMatrix4x4(0.0, 0.0, 1.0, -20.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)
print(curr_transform)

curr_transform.matrix()
# spike_raster_plt.ui.gx.

In [ ]:
spike_raster_plt.params.keys()

In [ ]:
# spike_raster_plt.params.spike_start_z # -10
spike_raster_plt.params.spike_end_z = -8.0 # -6.0

In [ ]:
spike_raster_plt.animation_time_step = 0.06

In [ ]:
spike_raster_plt.shift_animation_frame_val(-3)

In [ ]:
curr_spikes_df

# Test 3D Curve Plotting with SpikeRaster3D

In [ ]:
# temporal_to_spatial_transform_computation
np.shape(curr_sess.mua.firing_rate) # (1716893,)
np.shape(curr_sess.mua.time) # (1716893,)

yi = - # plot along the back wall in the y-dir
mua_pts = np.column_stack([x, np.full_like(x, yi), z])
plt = gl.GLLinePlotItem(pos=pts, color=pg.mkColor((i,n*1.3)), width=(i+1)/10., antialias=True)
w.addItem(plt)


    
